<a href="https://colab.research.google.com/github/dhaniel96/dnaiel/blob/master/hamoye_project_on_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Machine** **Learning**: Regression - Predicting Energy Efficiency of Buildings

In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
from geopy.distance import geodesic
from geopy.distance import lonlat, distance
df = pd.read_csv('energydata_complete.csv', delimiter = ',')

In [ ]:
#loading the set
df

In [ ]:
print(f'the number of rows in the dataset is {df.shape[0]}\nthe number of columns in the dataset is {df.shape[1]}')

In [ ]:
#checking if there  are missing values in the dataset
df.isnull().sum()

In [ ]:
#descriptive summary of the data
df.describe()

In [ ]:
#dropping the date and light columns
df = df.drop(['date','lights'],axis=1)
df.head()

In [59]:
#normalising our dataset to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


In [61]:
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
features_df = normalised_df.drop(columns=['Appliances'], axis=1)
heating_target = normalised_df['Appliances']

In [57]:
#splitting our dataset into the training and testing dataset. 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_df, heating_target, test_size=0.3, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
predicted_values = linear_model.predict(x_test)

In [ ]:
#MAE(Mean Absolute Error)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2)

In [ ]:
#Residual Sum of Squares (RSS)
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2)

In [ ]:
#Root Mean Square Error (RMSE)
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

In [ ]:
#R-Squared
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

In [ ]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(x_train, y_train)

In [ ]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

In [ ]:
linear_model = get_weights_df(linear_model, x_train, 'Linear_Model')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

In [ ]:
final_weights = pd.merge(linear_model, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights